In [1]:
# PIL is PILLOW, the updated Python Imaging Library
from PIL import Image
import os, sys
import queue

In [2]:
# constants of famous colors, represented as triples of RGB values
black = (0, 0, 0)
red = (255, 0, 0)
yellow = (0, 255, 0)
blue = (0, 0, 255)
white = (255, 255, 255)

# directions for BFS traverse
dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

In [3]:
# Euclidean norm; sense of how "big" or "bright"
def norm(pixel):
    x = pixel[0]
    y = pixel[1]
    z = pixel[2]
    return x*x + y*y + z*z

# detect if x, y, are in range of 2D array of size A x B
def valid(x, y, A, B):
    if x < 0 or x >= A or y < 0 or y >= B:
        return False
    return True

In [4]:
# choose the best letter for this particular block; currently only handles 7 x 7 circles
def get_letter(pixels, x1, y1, x2, y2, candidates):
    
    err = (x2-x1) * (y2 - y1)
    index = -1
    
    for k in candidates:
        
        name = "0" + str(k)
        if k < 10:
            name = "0" + name
        
        im_let = Image.open("letters/full" + name + ".jpg")
        #im_let = im_let.resize((x2-x1, x2-x1))
        let_map = im_let.load()
        
        # images are corrupted or sth; gotta set back to red/black contrast
        for i in range(im_let.size[0]):
            for j in range(im_let.size[1]):
                pxl = let_map[i,j]
                if norm(pxl) > 90*90:
                    let_map[i,j] = (255, 0, 0)
                else:
                    let_map[i,j] = (0, 0, 0)
                    
        # perform bfs on original image to fill in black surrounding circle into red
        q = queue.Queue()
        q.put((x1, y1))
        q.put((x2, y1))
        q.put((x1, y2))
        q.put((x2, y2))

        my_set = set()

        while not q.empty():
            coor = q.get()
            x = coor[0]
            y = coor[1]

            if coor in my_set:
                continue

            my_set.add(coor)

            pixels[x, y] = (255, 0, 0)

            for m in range(4):
                newx = x + dx[m]
                newy = y + dy[m]
                t = (newx, newy)
                if x1 <= newx and newx <= x2 and y1 <= newy and newy <= y2 and pixels[newx, newy] == (0, 0, 0) and t not in my_set:
                    q.put(t)
        #end bfs
        
        '''
        # get bounding box on black pixels
        X1 = 10000
        X2 = 0
        Y1 = 10000
        Y2 = 0
        for i in range(x1, x2 + 1):
            for j in range(y1, y2 + 1):
                if pixels[i, j] == (0, 0, 0):
                    X1 = min(i, X1)
                    X2 = max(i, X2)
                    Y1 = min(j, Y1)
                    Y2 = max(j, Y2)
                    
        print("SMALL BOUNDING BOX = ", X1, Y1, X2, Y2)
        '''
        
        img_1 = Image.new(im_let.mode, (x2-x1+1, y2-y1+1))
        p2 = img_1.load()
        for i in range(x1, x2+1):
            for j in range(y1, y2+1):
                p2[i-x1, j-y1] = pixels[i, j]
                
        img_1 = img_1.resize(im_let.size)
        p2 = img_1.load()
        
        # count errors
        temp_err = 0
        
        #for i in range(min(im_let.size[0], x2-x1)):
            #for j in range(min(im_let.size[1], y2-y1)):
        #for i in range(X2+1 - X1):
            #for j in range(Y2+1 - Y1):
        for i in range(img_1.size[0]):
            for j in range(img_1.size[1]):
                #pxl1 = pixels[x1 + i, y1 + j]
                pxl1 = p2[i, j]
                pxl2 = let_map[i, j]
                if pxl1 != pxl2:
                    temp_err += 1
        
        #print("temp err for ", chr(ord('A')  + k), " is ", temp_err)
        
        # which ever is least is the winner
        if temp_err < err:
            err = temp_err
            index = k
    
    # return the corresponding character
    return chr(ord('A') + index)


In [5]:
def bfs(pixels, points, x1, y1, x2, y2, search_color, new_color):
    q = queue.Queue()
    for point in points:
        if x1 <= point[0] and point[0] <= x2 and y1 <= point[1] and point[1] <= y2 and pixels[point[0], point[1]] == search_color:
            q.put(point)
    
    visited = set()
    while not q.empty():
        point = q.get()
        x = point[0]
        y = point[1]
        
        if not (x1 <= x and x <= x2) or not (y1 <= y and y <= y2):
            continue
        if point in visited:
            continue
            
        visited.add(point)
        
        pixels[x, y] = new_color
        
        for k in range(4):
            newx = x + dx[k]
            newy = y + dy[k]
            
            if x1 <= newx and newx <= x2 and y1 <= newy and newy <= y2 and pixels[newx, newy] == search_color:
                q.put((newx, newy))

In [6]:
def get_row_nums(pixels, x1, y1, x2, y2):
    ans = []
    for i in range(y1, y2+1):
        temp = 0
        for j in range(x1+1, x2+1):
            if pixels[j-1, i] == red and pixels[j, i] == black:
                temp += 1
        
        if temp != 0: #and (len(ans) == 0 or ans[-1] != temp):
            ans.append(temp)
    
    ans1 = []
    for i in range(2, len(ans)):
        if ans[i-2] == ans[i] and ans[i-1] == ans[i] and (len(ans1) == 0 or ans[i] != ans1[-1]):
            ans1.append(ans[i])
            
    return ans1
    
def get_col_nums(pixels, x1, y1, x2, y2):
    ans = []
    for i in range(x1, x2+1):
        temp = 0
        for j in range(y1+1, y2+1):
            if pixels[i, j-1] == red and pixels[i, j] == black:
                temp += 1
                
        if temp != 0: #and (len(ans) == 0 or ans[-1] != temp):
            ans.append(temp)
    
    ans1 = []
    for i in range(2, len(ans)):
        if ans[i-2] and ans[i] and ans[i-1] == ans[i] and (len(ans1) == 0 or ans[i] != ans1[-1]):
            ans1.append(ans[i])
            
    return ans1

def purify(pixels, x1, y1, x2, y2):
    for i in range(y1, y2+1):
        for j in range(x1, x2+1-4):
            pxl = [pixels[j, i], pixels[j+1, i], pixels[j+2, i], pixels[j+3, i], pixels[j+4, i]]
            if pxl[0] == red and pxl[4] == red and (pxl[1] == black or pxl[2] == black or pxl[3] == black):
                pixels[j+1, i] = red
                pixels[j+2, i] = red
                pixels[j+3, i] = red
            
            '''
            if pxl[0] == black and pxl[4] == black and (pxl[1] == red or pxl[2] == red or pxl[3] == red):
                pixels[j+1, i] = black
                pixels[j+2, i] = black
                pixels[j+3, i] = black
            '''
    for i in range(x1, x2+1):
        for j in range(y1, y2+1-4):
            pxl = [pixels[i, j], pixels[i, j+1], pixels[i, j+2], pixels[i, j+3], pixels[i, j+4]]
            if pxl[0] == red and pxl[4] == red and (pxl[1] == black or pxl[2] == black or pxl[3] == black):
                pixels[i, j+1] = red
                pixels[i, j+2] = red
                pixels[i, j+3] = red
            
            '''
            if pxl[0] == black and pxl[4] == black and (pxl[1] == red or pxl[2] == red or pxl[3] == red):
                pixels[i, j+1] = black
                pixels[i, j+2] = black
                pixels[i, j+3] = black
            '''
    for i in range(x1, x2+1):
        for j in range(y1, y2 + 1 - 3):
            pxl = [pixels[i, j], pixels[i, j+1], pixels[i, j+2], pixels[i, j+3]]
            if pxl[0] == black and pxl[3] == black and (pxl[1] == red or pxl[2] == red):
                pixels[i, j+1] = black
                pixels[i, j+2] = black
                pixels[i, j+3] = black
                
    for i in range(y1, y2+1):
        for j in range(x1, x2+1-2):
            pxl = [pixels[j, i], pixels[j+1, i], pixels[j+2, i]]
            if pxl[0] == black and pxl[2] == black and (pxl[1] == red):
                pixels[j+1, i] = black

In [73]:
# choose the best letter for this particular block
def get_letter2(pixels, x1, y1, x2, y2):
   
    row_nums = get_row_nums(pixels, x1, y1, x2, y2)
    col_nums = get_col_nums(pixels, x1, y1, x2, y2)
    
    print("")
    print(row_nums)
    print(col_nums)
    
    if row_nums == None or len(row_nums) == 0 or col_nums == None or len(col_nums) == 0:
        print("Nani")
        return None
    
    # 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
    # A B C D E F G H I J  K  L  M  N  O  P  Q  R  S  T  U  V  W  X  Y  Z
    
    if col_nums == [1, 3, 2]:
        return 'E'
    
    if col_nums == [1]:
        if len(row_nums) >= 2:
            if row_nums[-2] == 2 and row_nums[-1] == 1:
                return get_letter(pixels, x1, y1, x2, y2, (12, 20, 21, 24)) # M, U, V, Y
    
    if col_nums == [1, 2, 1]:
        if len(row_nums) >= 4 and row_nums[0:4] == [1, 2, 1, 2]:
            return 'A'
        
    if row_nums == [1]:
        if col_nums == [1]:
            #print("either L or T or I or J")
            res = get_letter(pixels, x1, y1, x2, y2, (8, 9, 11, 19))
            return res
        elif col_nums == [1, 2]:
            # C or F
            res = get_letter(pixels, x1, y1, x2, y2, (2, 5))
            return res
        elif col_nums == [1, 2, 1]:
            #print("either F or J") # or C
            #res = get_letter(pixels, x1, y1, x2, y2, (5, 9))
            #return res
            res = get_letter(pixels, x1, y1, x2, y2, (2, 5))
            return res
        elif col_nums == [1, 3]:
            return 'E'
        elif col_nums == [1, 3, 2]:
            return 'E'
        elif col_nums == [1, 3, 2, 1]:
            return 'E'
        elif col_nums == [1, 2, 3, 2]:
            res = get_letter(pixels, x1, y1, x2, y2, (4, 25))
            return res
        elif col_nums == [2]:
            return 'Z'
        elif col_nums == [2, 3, 2, 1]:
            return 'S'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [1, 2, 1]:
        if col_nums == [1]:
            return 'J'
        elif col_nums == [1, 2, 1]:
            #print("either D or O or P or Q")
            res = get_letter(pixels, x1, y1, x2, y2, (3, 14, 15, 16))
            return res
        elif col_nums == [1, 2, 3, 2]:
            return 'G'
        elif col_nums == [1, 2, 3, 2, 1]:
            res = get_letter(pixels, x1, y1, x2, y2, (16, 18, 6)) # Q or S or G
            return res
        elif col_nums == [2, 3, 2, 1]:
            return 'S'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [1, 2, 1, 2]:
        if col_nums == [1, 2, 1]:
            return 'A'
        elif len(col_nums) >= 4 and col_nums[0:4] == [1, 2, 1, 2]:
            return 'R'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [1, 2, 1, 2, 1]:
        if col_nums == [1, 2]:
            return 'C'
        elif len(col_nums) >= 2 and col_nums[0] == 1 and col_nums[1] == 3:
            return 'B'
        elif len(col_nums) >= 4 and col_nums[-4:] == [2, 3, 2, 1]:
            #print("either G or S")
            res = get_letter(pixels, x1, y1, x2, y2, (6, 18))
            return res
#        elif col_nums == [1, 3, 2, 1]:
#            return 'B'
        elif 3 in col_nums:
            res = get_letter(pixels, x1, y1, x2, y2, (1, 6, 18))
            return res
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [1, 3, 4, 3, 2]:
        if col_nums == [1]:
            return 'W'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [2]:
        if col_nums == [1]:
            return 'N'
    elif row_nums == [2, 1]:
        if col_nums == [1]:
            #print("either U or V or Y")
            res = get_letter(pixels, x1, y1, x2, y2, (20, 21, 24))
            return res
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [2, 1, 2]:
        if col_nums == [1]:
            return 'H'
        elif col_nums == [1, 2]:
            return 'K'
        elif len(col_nums) >= 3 and col_nums[0:3] == [1, 2, 1]:
            #print("either K or X")
            res = get_letter(pixels, x1, y1, x2, y2, (10, 23))
            return res
        elif col_nums == [2, 1, 2]:
            return 'X'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [2, 3]:
        if col_nums == [1]:
            return 'M'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [2, 3, 2]:
        '''
        if col_nums == [1, 2, 1, 2, 1]:
            return 'N'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
        '''
        res = get_letter(pixels, x1, y1, x2, y2, (12, 13)) #12, 13
        return res
    elif row_nums == [2, 4, 3, 2]:
        if col_nums == [1]:
            return 'M'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [2, 4, 3, 2, 1]:
        if col_nums == [1]:
            return 'M'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [3, 2]:
        if col_nums == [1]:
            return 'W'
        else:
            print("UNKNOWN")
            print(row_nums)
            print(col_nums)
    elif row_nums == [3, 4, 3, 2]:
        if col_nums == [1]:
            return 'W'
    else:
        print("UNKNOWN")
        print(row_nums)
        print(col_nums)
        
    
    return (row_nums, col_nums)

In [87]:
# start here! currently works for wb_00000.jpg and wb_00001.jpg, both 7 x 7 boards
im = Image.open("wb_00011.jpg")
#im.show()

# the images i work with are about 750 x 1334 pixels
my_width = 750
my_height = int(im.size[1] * (my_width / im.size[0]))

# resize the image
im = im.resize((my_width, my_height))

print(im.size)
pixelMap = im.load()

# 7 x 7 bounding box: x: 70, 675 y: 146, 1050
# one bubble has diameter of about 76-78 pixels
# red = (191, 100, 71)

# ngl i have no idea what count is
count = 0

# create a new image as not to modify the original image
imgNew = Image.new(im.mode, im.size)
pixelNew = imgNew.load()

# store coordinates of all character bubbles
bubble_coor = []

# bounding box of all character bubbles
minx = im.size[0]
miny = im.size[1]
maxx = 0
maxy = 0

# black out surroundings
for i in range(im.size[0]):
    for j in range(im.size[1]):
        
        if 70 <= i and i < 686 and 146 <= j and j < 1050:
            depth = norm(pixelMap[i,j])
            # turn white-ish background into black
            if depth >= 150000:
                pixelNew[i,j] = (0,0,0, 255)
                count += 1
            # turn bottom solution bubbles into white
            elif depth >= 120000:
                pixelNew[i,j] = (255, 255, 255, 255)
            # copy over board bubble colors
            else:
                pixelNew[i,j] = pixelMap[i,j]
        else:
            pixelNew[i,j] = (0, 0, 0, 255)

# get bounding box
for i in range(70 - 10, 686 + 10):
    for j in range(146 - 10, 800 + 10):
        pxl = pixelNew[i,j]
        if (not (pxl == (0,0,0))) and (not (pxl == (255, 255, 255))):
            minx = min(i, minx)
            miny = min(j, miny)
            maxx = max(i, maxx)
            maxy = max(j, maxy)
       
# remove that annoying thing in the top right corner, which is white, and turn black
for i in range(minx-1, maxx+1):
    for j in range(miny-1, maxy+1):
        pxl = pixelNew[i,j]
        if pxl == (255, 255, 255):
            pixelNew[i,j] = (0, 0, 0, 255)
            
# turn board bubbles into red
for i in range(minx-1, maxx+1):
    for j in range(miny-1, maxy+1):
        pxl = pixelNew[i,j]
        if (not (pxl == (0,0,0))) and (not (pxl == (255, 255, 255))):
            pixelNew[i,j] = (255, 0, 0, 255)

# make a copy
imgNew_red = Image.new(im.mode, im.size)
pixelNew_red = imgNew_red.load()

for i in range(im.size[0]):
    for j in range(im.size[1]):
        pixelNew_red[i,j] = pixelNew[i,j]

# bounding green box around each bubble in board
for i in range(minx-1, maxx+1):
    for j in range(miny-1, maxy+1):
        pxl = pixelNew[i,j]
        if pxl == red:
            #x1 y1 x2 y2 are coordinates of bounding box
            x1 = i
            y1 = j
            x2 = i
            y2 = j
            
            x = i
            y = j
            
            in_list = set()
            
            q = queue.Queue()
            q.put((x, y))
            
            while not q.empty():
                coor = q.get()
       #         print("coor = ", coor)
                pixelNew[coor[0], coor[1]] = yellow
                
                x1 = min(coor[0], x1)
                x2 = max(coor[0], x2)
                y1 = min(coor[1], y1)
                y2 = max(coor[1], y2)
                
                if (coor[0]+1, coor[1]) not in in_list and pixelNew[coor[0] + 1, coor[1]] == red:
                    q.put((coor[0] + 1, coor[1]))
                    in_list.add((coor[0]+1, coor[1]))
                if (coor[0]-1, coor[1]) not in in_list and pixelNew[coor[0] - 1, coor[1]] == red:
                    q.put((coor[0] - 1, coor[1]))
                    in_list.add((coor[0]-1, coor[1]))
                if (coor[0], coor[1] + 1) not in in_list and pixelNew[coor[0], coor[1] + 1] == red:
                    q.put((coor[0], coor[1] + 1))
                    in_list.add((coor[0], coor[1] + 1))
                if (coor[0], coor[1] - 1) not in in_list and pixelNew[coor[0], coor[1] - 1] == red:
                    q.put((coor[0], coor[1] - 1))
                    in_list.add((coor[0], coor[1] - 1))
            
            #print(x2 - x1, y2 - y1)
            
            if (x2-x1) * (y2 - y1) < 100:
                # area is too small; for some reason this occurred; must be an error
                continue
            
            t = ((x1, y1), (x2, y2))
            print("gonna add t =", t)
            bubble_coor.append(t)
            
            for i in range(x1, x2):
                for j in range(y1, y2):
                    pixelNew[i, j] = (0, 255, 0)

#imgNew_red.show()

# get / save image files
# 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
# A B C D E F G H I J  K  L  M  N  O  P  Q  R  S  T  U  V  W  X  Y  Z
#relevant = {1:"18", 3:"11", 4:"20", 5:"08", 6:"13", 7:"03", 9:"05", 10:"13", 14:"10", 15:"24",20:"01", 22:"00", 23:"19", 25:"17", 29:"20", 35:"02", 40:"07", 41:"14", 43:"06"}
'''
for k in range(len(bubble_coor)):
    top_left = bubble_coor[k][0]
    bot_right = bubble_coor[k][1]
    x1 = top_left[0]
    y1 = top_left[1]
    x2 = bot_right[0]
    y2 = bot_right[1]
    
    small = min(x2 - x1, y2 - y1)
    
    img_let = Image.new(im.mode, (small, small))
    pxl_let = img_let.load()
    
    for i in range(small):
        for j in range(small):
            pxl_let[i,j] = pixelNew_red[x1 + i, y1 + j]

#    img_let.show()
    print("k = ", k)
    
    my_str = "letters/" + str(k) + ".jpg"
    img_let.save(my_str)


#imgNew_red.show()
#imgNew.show()
'''

# get how big the actual board is; i'm also thinking of doing sth else for this
side_len = 1
while side_len * side_len < len(bubble_coor):
    side_len += 1

print("side len = ", side_len)
print("bubble coor len = ", len(bubble_coor))
    
board = []
for i in range(side_len):
    temp = []
    for j in range(side_len):
        temp.append(' ')
    board.append(temp)
    
# fill in squares to boxes
for i in range(len(bubble_coor)):
    top_left = bubble_coor[i][0]
    bot_right = bubble_coor[i][1]
    x1 = top_left[0]
    y1 = top_left[1]
    x2 = bot_right[0]
    y2 = bot_right[1]
    
    for i in range(x1, x2+1):
        for j in range(y1, y2+1):
            pxl = pixelNew_red[i,j]
            if norm(pxl) > 100*100:
                pixelNew_red[i,j] = (255, 0, 0)
            else:
                pixelNew_red[i,j] = (0, 0, 0)
    
    # turn character in circle into character in square
    points = ((x1, y1), (x1, y2), (x2, y1), (x2, y2))
    bfs(pixelNew_red, points, x1, y1, x2, y2, black, red)
    
    purify(pixelNew_red, x1, y1, x2, y2)

# minimum error
for i in range(len(bubble_coor)):
    top_left = bubble_coor[i][0]
    bot_right = bubble_coor[i][1]
    x1 = top_left[0]
    y1 = top_left[1]
    x2 = bot_right[0]
    y2 = bot_right[1]
    
    bubble_len = (x2-x1) + 10
    col = round((x1 - minx) / bubble_len)
    row = round((y1 - miny) / bubble_len)
    
    #if(row != 4 or col != 0):
    #    continue
    
    result = get_letter2(pixelNew_red, x1, y1, x2, y2)
    
    # since row / col are swapped in Pillow,
#    col = i // side_len
#    row = i % side_len

    #print("x1 y1 x2 y2 =", x1, y1, x2, y2)
    #print("size = ", (x2-x1), (y2-y1))

    
    print("access [", col, row, "]")
    
    print("temp_result =", result)
    board[row][col] = result

for i in range(len(board)):
    print(board[i])

#imgNew.show()

imgNew_red.show()


print("count =", count)
print("top left = (", minx, ",", miny, ") and bot right = (", maxx, ",", maxy, ")")
print("side_len =", side_len)
print("board =", board)
print("len of result =", len(board))


(750, 1334)
gonna add t = ((71, 234), (147, 310))
gonna add t = ((71, 322), (147, 398))
gonna add t = ((71, 409), (147, 485))
gonna add t = ((71, 497), (147, 573))
gonna add t = ((71, 584), (148, 661))
gonna add t = ((71, 672), (148, 748))
gonna add t = ((159, 146), (235, 223))
gonna add t = ((159, 234), (235, 310))
gonna add t = ((159, 321), (235, 398))
gonna add t = ((159, 409), (235, 485))
gonna add t = ((159, 497), (235, 573))
gonna add t = ((159, 585), (235, 661))
gonna add t = ((159, 672), (235, 748))
gonna add t = ((246, 146), (323, 223))
gonna add t = ((246, 234), (323, 310))
gonna add t = ((246, 322), (323, 398))
gonna add t = ((246, 409), (323, 485))
gonna add t = ((246, 497), (323, 573))
gonna add t = ((246, 584), (323, 660))
gonna add t = ((246, 672), (322, 748))
gonna add t = ((334, 146), (410, 223))
gonna add t = ((334, 234), (410, 310))
gonna add t = ((334, 322), (410, 398))
gonna add t = ((334, 409), (410, 485))
gonna add t = ((334, 497), (410, 573))
gonna add t = ((334

In [ ]:
'''
# turn letters in circles into letters in squares
#invalid = [9, 16, 21, 22, 23, 25]
invalid = [17]
#toAdd = [12, 15]

for z in range(26):
    if z not in invalid:
        continue
        
    name = "0" + str(z)
    if z < 10:
        name = "0" + name
        
    letter = Image.open("letters/" + name + ".jpg")
    board = letter.load()
    
    full_letter = Image.new(im.mode, letter.size)
    newboard = full_letter.load()

    # images are corrupted or sth; gotta set back to red/black contrast
    for i in range(letter.size[0]):
        for j in range(letter.size[1]):
            pxl = board[i,j]
            if norm(pxl) > 100*100:
                board[i,j] = (255, 0, 0)
            else:
                board[i,j] = (0, 0, 0)
    
    for i in range(letter.size[0]):
        for j in range(letter.size[1]):
            newboard[i, j] = board[i, j]
                
    q = queue.Queue()
    q.put((0, 0))
    q.put((letter.size[0] - 1, 0))
    q.put((0, letter.size[1] - 1))
    q.put((letter.size[0] - 1, letter.size[1] - 1))
    
    my_set = set()
    
    while not q.empty():
        coor = q.get()
        x = coor[0]
        y = coor[1]
        
        if coor in my_set:
            continue
            
        my_set.add(coor)
        
        newboard[x, y] = (255, 0, 0)
        
        for k in range(4):
            newx = x + dx[k]
            newy = y + dy[k]
            t = (newx, newy)
            if valid(newx, newy, letter.size[0], letter.size[1]) and board[newx, newy] == (0, 0, 0) and t not in my_set:
                q.put(t)
        
    full_letter.save("letters/full" + name + ".jpg")
    full_letter.show()
'''

In [36]:
for let in range(26):
    name = "0" + str(let)
    if let < 10:
        name = "0" + name
        
    img = Image.open("letters/full" + name + ".jpg")
    pixels = img.load()
    
    # images are corrupted or sth; gotta set back to red/black contrast
    for i in range(img.size[0]):
        for j in range(img.size[1]):
            pxl = pixels[i,j]
            if norm(pxl) > 90*90:
                pixels[i,j] = (255, 0, 0)
            else:
                pixels[i,j] = (0, 0, 0)
    
    #img.show()
    
    tup = get_letter2(pixels, 0, 0, img.size[0] - 1, img.size[1] - 1)
    
    my_char = chr(ord('A') + let)
    print("char " + str(my_char) + " received ", tup)
    

char A received  A
char B received  B
char C received  C
char D received  D
char E received  E
char F received  F
char G received  G
char H received  H
char I received  I
char J received  J
char K received  K
char L received  L
char M received  M
char N received  N
char O received  O
char P received  P
char Q received  Q
char R received  R
char S received  S
char T received  T
char U received  U
char V received  V
char W received  W
char X received  X
char Y received  Y
char Z received  Z
